In [3]:
!pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.4 MB 1.1 MB/s 
     |████████████████████████████████| 4.1 MB 42.2 MB/s 
     |████████████████████████████████| 8.8 MB 48.8 MB/s 
     |████████████████████████████████| 125 kB 75.7 MB/s 
     |████████████████████████████████| 468 kB 65.8 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.48.1
    Uninstalling grpcio-1.48.1:
      Successfully uninstalled grpcio-1.48.1


In [54]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from filelock import FileLock
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
import ray
from ray import tune
from ray.air import session
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler


In [55]:
def load_data(data_dir="./data"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    with FileLock(os.path.expanduser("~/.data.lock")):
        trainset = torchvision.datasets.CIFAR10(
            root=data_dir, train=True, download=True, transform=transform)

        testset = torchvision.datasets.CIFAR10(
            root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [56]:
class Net(nn.Module):
    def __init__(self, l1=120, l2=84):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, l1)
        self.fc2 = nn.Linear(l1, l2)
        self.fc3 = nn.Linear(l2, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [57]:
def train_cifar(config):
    net = Net(config["l1"], config["l2"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    # To restore a checkpoint, use `session.get_checkpoint()`.
    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
           model_state, optimizer_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    data_dir = os.path.abspath("./data")
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True,
        num_workers=8)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("my_model", exist_ok=True)
        torch.save(
            (net.state_dict(), optimizer.state_dict()), "my_model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("my_model")
        session.report({"loss": (val_loss / val_steps), "accuracy": correct / total}, checkpoint=checkpoint)
    print("Finished Training")

In [58]:
def test_best_model(best_result):
    best_trained_model = Net(best_result.config["l1"], best_result.config["l2"])
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_result.checkpoint.to_directory(), "checkpoint.pt")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    trainset, testset = load_data()

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = best_trained_model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    print("Best trial test set accuracy: {}".format(correct / total))

In [59]:
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16]),
}

In [72]:
num_samples=10
max_num_epochs=10
gpus_per_trial=1

config = {
    "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}
scheduler = ASHAScheduler(
    max_t=max_num_epochs,
    grace_period=1,
    reduction_factor=2
    )
reporter = CLIReporter(
    # parameter_columns=["l1", "l2", "lr", "batch_size"],
    metric_columns=["loss", "accuracy", "training_iteration"])
   
tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train_cifar),
        resources={"cpu": 2, "gpu": gpus_per_trial}
    ),
    tune_config=tune.TuneConfig(
        metric="loss",
        mode="min",
        scheduler=scheduler,
        num_samples=num_samples,
            
    ),
    param_space=config
)
results = tuner.fit()
   
best_result = results.get_best_result("loss", "min")

print("Best trial config: {}".format(best_result.config))
print("Best trial final validation loss: {}".format(
    best_result.metrics["loss"]))
print("Best trial final validation accuracy: {}".format(
    best_result.metrics["accuracy"]))

test_best_model(best_result)

Trial name,status,loc,batch_size,l1,l2,lr,iter,total time (s),loss,accuracy
train_cifar_13a1d_00000,TERMINATED,172.28.0.2:5760,16,16,32,0.000365922,10,218.574,1.33077,0.5217
train_cifar_13a1d_00001,TERMINATED,172.28.0.2:5760,8,256,16,0.0356145,1,37.0142,2.31333,0.1015
train_cifar_13a1d_00002,TERMINATED,172.28.0.2:5760,8,32,16,0.000423862,10,324.411,1.17589,0.587
train_cifar_13a1d_00003,TERMINATED,172.28.0.2:5760,8,256,128,0.00159949,10,323.686,1.19103,0.6181
train_cifar_13a1d_00004,TERMINATED,172.28.0.2:5760,16,64,32,0.0236986,2,49.2114,1.78205,0.3483
train_cifar_13a1d_00005,TERMINATED,172.28.0.2:5760,4,64,16,0.00117412,10,563.124,1.19332,0.5996
train_cifar_13a1d_00006,TERMINATED,172.28.0.2:5760,2,256,4,0.00361404,1,108.695,2.30432,0.1009
train_cifar_13a1d_00007,TERMINATED,172.28.0.2:5760,2,16,4,0.0492786,1,105.475,2.38906,0.0962
train_cifar_13a1d_00008,TERMINATED,172.28.0.2:5760,4,64,64,0.00099349,10,558.11,1.21816,0.5929
train_cifar_13a1d_00009,TERMINATED,172.28.0.2:5760,2,8,64,0.00172191,2,201.886,1.65788,0.3894


(train_cifar pid=5760) Downloading https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00000_0_batch_size=16,l1=16,l2=32,lr=0.0004_2022-09-30_02-03-29/data/cifar-10-python.tar.gz


100%|██████████| 170498071/170498071 [00:02<00:00, 76289799.08it/s]


(train_cifar pid=5760) Extracting /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00000_0_batch_size=16,l1=16,l2=32,lr=0.0004_2022-09-30_02-03-29/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00000_0_batch_size=16,l1=16,l2=32,lr=0.0004_2022-09-30_02-03-29/data
(train_cifar pid=5760) Files already downloaded and verified


(train_cifar pid=5760) /usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
(train_cifar pid=5760)   cpuset_checked))


...

(train_cifar pid=5760) Extracting /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00008_8_batch_size=4,l1=64,l2=64,lr=0.0010_2022-09-30_02-32-21/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00008_8_batch_size=4,l1=64,l2=64,lr=0.0010_2022-09-30_02-32-21/data
(train_cifar pid=5760) Files already downloaded and verified
(train_cifar pid=5760) [1,  2000] loss: 2.206
(train_cifar pid=5760) [1,  4000] loss: 0.915
(train_cifar pid=5760) [1,  6000] loss: 0.549
(train_cifar pid=5760) [1,  8000] loss: 0.389
(train_cifar pid=5760) [1, 10000] loss: 0.300
Result for train_cifar_13a1d_00008:
  accuracy: 0.4526
  date: 2022-09-30_02-33-25
  done: false
  experiment_id: 73c5e13b7be242449a3cb20da1539a18
  hostname: a758025fc9b4
  iterations_since_restore: 1
  loss: 1.4963509201049805
  node_ip: 172.28.0.2
  pid: 5760
  should_checkpoint: true
  time_since_restore: 63.895647048950195
  time_this_iter_s: 63.895647048950195
  time_t

100%|██████████| 170498071/170498071 [00:02<00:00, 79170138.26it/s]


(train_cifar pid=5760) Extracting /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00009_9_batch_size=2,l1=8,l2=64,lr=0.0017_2022-09-30_02-41-39/data/cifar-10-python.tar.gz to /root/ray_results/train_cifar_2022-09-30_02-03-28/train_cifar_13a1d_00009_9_batch_size=2,l1=8,l2=64,lr=0.0017_2022-09-30_02-41-39/data
(train_cifar pid=5760) Files already downloaded and verified
(train_cifar pid=5760) [1,  2000] loss: 2.201
(train_cifar pid=5760) [1,  4000] loss: 0.966
(train_cifar pid=5760) [1,  6000] loss: 0.616
(train_cifar pid=5760) [1,  8000] loss: 0.449
(train_cifar pid=5760) [1, 10000] loss: 0.353
(train_cifar pid=5760) [1, 12000] loss: 0.286
(train_cifar pid=5760) [1, 14000] loss: 0.246
(train_cifar pid=5760) [1, 16000] loss: 0.214
(train_cifar pid=5760) [1, 18000] loss: 0.188
(train_cifar pid=5760) [1, 20000] loss: 0.164
Result for train_cifar_13a1d_00009:
  accuracy: 0.4064
  date: 2022-09-30_02-43-23
  done: false
  experiment_id: 73c5e13b7be242449a3cb20da1539a18
  

2022-09-30 02:45:01,703	INFO tune.py:759 -- Total run time: 2492.85 seconds (2492.70 seconds for the tuning loop).


Best trial config: {'l1': 32, 'l2': 16, 'lr': 0.00042386219608474783, 'batch_size': 8}
Best trial final validation loss: 1.175886014330387
Best trial final validation accuracy: 0.587
Files already downloaded and verified
Files already downloaded and verified
Best trial test set accuracy: 0.5855
